<a id='top'></a>
# Using the API
In this notebook, we will see how to use *Spotipy* in Python to navigate the Spotify API.

---

## Prerequisites

>I. [Setting up the API Connection](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/01SettingUptheAPIConnection.ipynb)

---

## Table of Contents

> I. [Connect to Spotify](#ConnectToSpotify)<br>
> II. [The Search Function](#TheSearchFunction)<br>
> III. [Performing a Search](#PerformSearch)<br>
> IV. [Parsing the JSON](#ParsetheJSON)

---

<a id='ConnectToSpotify'></a>

### Connect to Spotify
Let's create a function that will conveniently get us connected to the Spotify API using the method outlined in the [Setting Up the API Connection](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/01SettingUptheAPIConnection.ipynb).

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def connect_to_spotify():
    sp_client_id = input("Enter your Spotify Client ID: ")
    sp_client_secret = input("Enter your Spotify Secret Token: ")
    
    credentials = SpotifyClientCredentials(client_id = sp_client_id,
                                           client_secret = sp_client_secret)
    
    sp = spotipy.Spotify(client_credentials_manager = credentials)
    
    print("Connected to Spotify!")

    return sp

In [2]:
sp = connect_to_spotify()

Enter your Spotify Client ID:  ed9307841d3542df8819aec9a4f0ec84
Enter your Spotify Secret Token:  8208717955574be6a24163ed59675094


Connected to Spotify!


<a name='TheSearchFunction'></a>
<br>

### The Search Function
It all starts with a search!<br>
Spotify assigns each track and artist a unique resource identifier (URI), which is essential for retrieving specific information about them. We obtain these URIs by using the <code>search</code> method from our connection object, <code>sp</code>. For detailed information on the <code>search</code> feature, please refer to the *[Spotipy documentation](https://spotipy.readthedocs.io/en/2.24.0/#spotipy.client.Spotify.search)*.<br>
<br>

For this project, we will be performing a **track** search by setting the <code>type</code> parameter to **'track'**. Here is an example of the syntax:<br>
<br>

<code>sp.search('Santa Baby', type = 'track')</code><br>
<br>

There are three reasons for utilizing search the type **'track'** on this project:<br>
1. We need are modeling songs, so we need a track URI.
1. We can obtain both the track and artist URI from a track search, if needed.
1. We can add the name of the artist to the search term.<br>

The search will always return the specified number of results (the <code>limit</code> parameter, default 10) regardless of how good a match the result is. To ensure the first result is the correct result, that third item in the list is important.

<a id='PerformSearch'></a>
<br>

### Performing a Search
Assume we want the URI for Kelly Clarkson's version of the Christmas song <i>Santa Baby</i>. Without specifying the artist name and just executing a search for the track name we get the following results:<br>
*(Ignore syntax for the time being. We will cover this next.)*

In [3]:
for item in sp.search('Santa Baby', type = 'track', limit = 15)['tracks']['items']:
    print(f'Artist: {item['artists'][0]['name']}')
    print(f'Track: {item['name']}')
    print('-' * 30)

Artist: Eartha Kitt
Track: Santa Baby
------------------------------
Artist: Eartha Kitt
Track: Santa Baby (with Henri René & His Orchestra)
------------------------------
Artist: Sleighbelle
Track: Santa Baby
------------------------------
Artist: Michael Bublé
Track: Santa Baby
------------------------------
Artist: Ariana Grande
Track: Santa Baby
------------------------------
Artist: Madonna
Track: Santa Baby
------------------------------
Artist: Taylor Swift
Track: Santa Baby
------------------------------
Artist: Kellie Pickler
Track: Santa Baby
------------------------------
Artist: SNAP!
Track: The Power
------------------------------
Artist: Matt Marantz Quartet
Track: Santa Baby
------------------------------
Artist: Kelly Clarkson
Track: Santa Baby
------------------------------
Artist: Milli Vanilli
Track: Girl You Know It's True
------------------------------
Artist: Michael Bublé
Track: Santa Baby
------------------------------
Artist: Kelly Clarkson
Track: Santa, Can’t 

<br>Notice I had to set the <code>limit</code> to 15 just to find our desired track. If this was the only way to retrieve that track URI, we would have to add some frustrating layers to our processing. However, we can avoid all that and ensure that we get the correct result by informing both the artist and track name in the search term as follows:

In [4]:
# The spacing is important!
search_term = f'artist:Kelly Clarkson track:Santa Baby'

# Let's just see what the first result returns
search_result = sp.search(search_term, type = 'track')['tracks']['items'][0]

print(f'Artist: {search_result['artists'][0]['name']}')
print(f'Track: {search_result['name']}')

Artist: Kelly Clarkson
Track: Santa Baby


<a id='ParsetheJSON'></a>
<br>

### Parsing the JSON
Now that we know how to use <code>search</code>, let's see how to get what we need from the result.<br>
As I'm sure you've already gathered, the results of the search are in JSON format. We will be examining how to retrieve *very specific* items from the result. If you are unfamiliar with JSON dictionaries I highly recommend giving [this](https://en.wikipedia.org/wiki/JSON) a quick read.<br>
I will be displaying the JSON results in a data frame format for readability.<br>
<br>
Before proceeding let's store the search results of <i>Santa Baby</i> into a variable.

In [5]:
search_results = sp.search('Santa Baby', limit = 15)

<br>

#### Layer 1: 'tracks'
With 15 results specificied, we would expect the length of our dictionary to be 15. However, it is not:

In [6]:
len(search_results)

1

<br>

This is because everything the very first term the dictionary, **'tracks'**, stores information regarding the query itself. Here's a better view:

In [7]:
import pandas as pd

pd.DataFrame(search_results)

,tracks
href,https://api.spotify.com/v1/search?query=Withou...
items,"[{'album': {'album_type': 'album', 'artists': ..."
limit,15
next,https://api.spotify.com/v1/search?query=Withou...
offset,0
previous,None
total,991


To get what we need, we need to go down another layer.

<br>

#### Layer 2: 'items'
From the above data frame we see that the information we need appears in **'items'**:

In [8]:
len(search_results['tracks']['items'])

15

<br>

That's what we want to see. This means that any time we perform a search, we can immediately go down two layers to **'items'**.

In [9]:
# Redo search result, except parse JSON down to 'items'
search_results = sp.search('Santa Baby', limit = 15)['tracks']['items']

# View search results (first 5)
pd.DataFrame(search_results).head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,206506,False,{'isrc': 'USRC15302564'},{'spotify': 'https://open.spotify.com/track/59...,https://api.spotify.com/v1/tracks/59hKzj0h6Elh...,59hKzj0h6ElhQv8rBqvpUz,False,Santa Baby,41,https://p.scdn.co/mp3-preview/142f12b53eb0cbec...,9,track,spotify:track:59hKzj0h6ElhQv8rBqvpUz
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,206066,False,{'isrc': 'USBB10101085'},{'spotify': 'https://open.spotify.com/track/1f...,https://api.spotify.com/v1/tracks/1foCxQtxBweJ...,1foCxQtxBweJtZmdxhEHVO,False,Santa Baby (with Henri René & His Orchestra),59,https://p.scdn.co/mp3-preview/0103100b0fab715e...,13,track,spotify:track:1foCxQtxBweJtZmdxhEHVO
2,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,120592,False,{'isrc': 'QZTB72300934'},{'spotify': 'https://open.spotify.com/track/0K...,https://api.spotify.com/v1/tracks/0Kj9Ft2fbVqS...,0Kj9Ft2fbVqSyulX2tOBS5,False,Santa Baby,39,https://p.scdn.co/mp3-preview/3da43f651229bfe2...,1,track,spotify:track:0Kj9Ft2fbVqSyulX2tOBS5
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,231626,False,{'isrc': 'USRE11100706'},{'spotify': 'https://open.spotify.com/track/3m...,https://api.spotify.com/v1/tracks/3m2gfwcxl77o...,3m2gfwcxl77ojJTWH3wZkb,False,Santa Baby,36,https://p.scdn.co/mp3-preview/d9d4755e733e0d55...,7,track,spotify:track:3m2gfwcxl77ojJTWH3wZkb
4,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,170933,False,{'isrc': 'USUM71318549'},{'spotify': 'https://open.spotify.com/track/6Y...,https://api.spotify.com/v1/tracks/6YJdPrH3i2PO...,6YJdPrH3i2POzu7hdHIRrb,False,Santa Baby,53,None,4,track,spotify:track:6YJdPrH3i2POzu7hdHIRrb


<br>

This is where we find our track URI (**'uri'**). Knowing how to search for a specific track and artist, how to limit our search results, and how to get down to this layer, we can now easily retrieve this:

In [10]:
search_term = f'artist:Kelly Clarkson track:Santa Baby'

# Even though we're only specifying 1 result, we still have to specify the index of that item, [0]
specific_search_result = sp.search(search_term, type = 'track', limit = 1)['tracks']['items'][0]

print(f'Track URI: {specific_search_result['uri']}')

Track URI: spotify:track:4sZBHxlO97cnA7SUbNjvNy


<br>

#### Layer 3: 'artists'
Even though we feel good about using the song and artist name in the search term, we are still going to want to be sure we got the right artist. For this, we need to go down one more layer to **'artists'**.

In [11]:
pd.DataFrame(specific_search_result['artists'])

,external_urls,href,id,name,type,uri
0,{'spotify': 'https://open.spotify.com/artist/3...,https://api.spotify.com/v1/artists/3BmGtnKgCSG...,3BmGtnKgCSGYIUhmivXKWX,Kelly Clarkson,artist,spotify:artist:3BmGtnKgCSGYIUhmivXKWX


<br>

There's what we need in **'name'**:

In [12]:
# Again, must specify the specific item by using the index number, [0]
print(f'Artist Name: {specific_search_result['artists'][0]['name']}')

Artist Name: Kelly Clarkson


<br>

<br>

#### Put It All Together
Let's do one last search in which we put it all together to get the track name, track URI, and artist name.

In [13]:
# Create search term
search_term = f'artist:Beatles track:Hello Goodbye'

# Store result of search
search_result = sp.search(search_term, type = 'track', limit = 1)['tracks']['items'][0]

# Store track name as returned by Spotify
track_title = search_result['name']

# Store track URI
track_uri = search_result['uri']

# Store artist name 
track_artist = search_result['artists'][0]['name']


print(f'Artist:\t\t{track_artist}')
print(f'Track:\t\t{track_title}')
print(f'Track URI:\t{track_uri}')

Artist:		The Beatles
Track:		Hello, Goodbye - Remastered 2009
Track URI:	spotify:track:0vZ97gHhemKm6c64hTfJNA


<br>

In the next notebook, [Spotify Audio Data Insights](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/03SpotifyAudioDataInsights.ipynb), we will learn how to use this information to retrieve track features and analyses.

<a href='#top'>Back to Top</a>